# Extracting symbols from image using `OpenCV`

In [3]:
import os
import cv2
import numpy as np
from functools import cmp_to_key
%matplotlib inline
from IPython.display import display, Image

In [4]:
def extractSymbols(imgOrig, showSteps = False):
    debugImgSteps = []
    imgGray = cv2.cvtColor(imgOrig,cv2.COLOR_BGR2GRAY)
    imgFiltered = cv2.medianBlur(imgGray, 5)
    debugImgSteps.append(imgFiltered)
    
    imgCanny = cv2.Canny(imgFiltered, 50,180)
    debugImgSteps.append(imgCanny)

    kernel = np.ones((5,5), np.uint8)
    imgDilated = cv2.dilate(imgCanny, kernel, iterations=5)
    debugImgSteps.append(imgDilated)

    contours, _= cv2.findContours(imgDilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    boundingBoxes = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        boundingBoxes.append((x,y,w,h))

    global rowsG
    rowsG, _, _ = imgOrig.shape
    key_leftRightTopBottom = cmp_to_key(leftRightTopBottom)
    boundingBoxes = sorted(boundingBoxes, key=key_leftRightTopBottom)

    symbols = []
    for (i, box) in enumerate(boundingBoxes):
        x,y,w,h = box
        mathSymbol = imgOrig[y:y+h, x:x+w]
        mathSymbol = cv2.resize(mathSymbol, (45,45), interpolation=cv2.INTER_AREA)
        symbols.append(mathSymbol)
        debugImgSteps.append(mathSymbol)
        image_name= "output_shape_number_" + str(i+1) + ".jpg"
        cv2.imwrite(image_name, mathSymbol)

    if showSteps:
        dispImages(debugImgSteps)

    return symbols
        

In [5]:
def leftRightTopBottom(tup1, tup2):
    x1, y1, _, _ = tup1
    x2, y2, _, _ = tup2
    rows = rowsG
    yRegion1, yRegion2 = -1, -1

    for i in range(8):
        if y1 < rows/8 + rows*(i/8):
            yRegion1 = i
            break
    else:
        if yRegion1 == -1:
            yRegion1 = 8

    for i in range(8):
        if y2 < rows/8 + rows*(i/8):
            yRegion2 = i
            break
    else:
        if yRegion2 == -1:
            yRegion2 = 8
    
    if yRegion1 < yRegion2:
        return -1
    elif yRegion2 < yRegion1:
        return 1
    elif x1 <= x2:
        return -1
    else:
        return 1


In [6]:
def dispImages(imgs):
    for img in imgs:
        cv2.imshow('Image', img)
        cv2.waitKey(0)
    else:
        cv2.destroyAllWindows()

In [7]:
img = cv2.imread('tests/testMath5.png')
symbols = extractSymbols(img, showSteps=False)
dispImages(symbols)

# Reading the kaggle [dataset](https://www.kaggle.com/datasets/xainano/handwrittenmathsymbols?resource=download)

In [8]:
def loadData(dataDir):
    FARAH = {}
    imgs = []
    labels = []
    for key, value in FARAH.items():
        path = os.path.join(dataDir, key)
        labels.append(value)
        for img in os.listdir(path):
            try:
                print()
            except Exception as e:
                print(e)    

# DICTIONARY

In [10]:
dic = {
    "-": "-",
    "!": "!",
    "(": "( \,",
    ")": ") \,",
    ",": "$,$",
    "[": "[ \,",
    "]": "] \,",
    "{": "\{",
    "}": "\}",
    "+": "+",
    "=": "=",
    "0": "0",
    "1": "1",
    "2": "2",
    "3": "3",
    "4": "4",
    "5": "5",
    "6": "6",
    "7": "7",
    "8": "8",
    "9": "9",
    "A": "\mathbb{A}",
    "alpha": "\alpha",
    "b": "b",
    "beta": "\beta",
    "C": "\C",
    "cos": "\cos",
    "d": "d",
    "Delta": "\Delta",
    "div": "\div",
    "e": "exp()",
    "exists": "\exists",
    "f": "f",
    "forall": "\forall",
    "forward_slash": "/",
    "G": "\G",
    "gamma": "\gamma",
    "geq": "\geq",
    "gt": ">",
    "H": "\H",
    "i": "i",
    "in": " \in",
    "infty": "\infty",
    "int": "\int",
    "j": "j",
    "k": "k",
    "l": "l",
    "lambda": "\lambda",
    "ldots": "\ldots",
    "leq": " \le",
    "lim": "\lim",
    "log": "\log",
    "lt": "<",
    "M": "\M",
    "mu": "\mu",
    "N": "\N",
    "neq": "\neq",
    "o": "\O",
    "p": "p",
    "phi": "\Phi",
    "pi": "\Pi",
    "pm": "\pm",
    "q": "q",
    "R": "\R",
    "rightarrow": "\rightarrow",
    "S": "\S",
    "sigma": "\sigma",
    "sin": "\sin",
    "sum": "\sum",
    "T": "\T",
    "tan": "\tan",
    "theta": "\theta",
    "times": "\times",
    "u": "u",
    "v": "v",
    "w": "w",
    "X": "\X",
    "y": "y",
    "z":"z"
}